# Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os

# Classification using pre-trained convolutional neural networks

In [ ]:
pip install split-folders

In [ ]:
!wget https://github.com/davidmallasen/LiveChess2FEN/releases/download/v0.1.0/ChessPiecesDataset.zip

!unzip /kaggle/working/ChessPiecesDataset.zip

In [ ]:
import splitfolders
splitfolders.ratio('./ChessPiecesDataset', output = "output", seed = 1, ratio = (.8, .1, .1))

In [ ]:
import os
base_dir = './output/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

from tensorflow.keras.optimizers import SGD
batch_size = 256
opt = SGD(learning_rate=0.01, momentum=0.9)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=5,
        # width_shift_range=0.1,
        # height_shift_range=0.1,
        rescale=1./255,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from os import path
# Define the stopping callbacks
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 20)

## VGG16

In [ ]:
shape = (224, 224)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)

from keras.applications.vgg16 import VGG16

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3)) 
 
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(1000, activation='relu')(x) # number of layers and units are hyperparameters, as usual
predictions = Dense(13, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model_VGG16 = Model(inputs=base_model.input, outputs=predictions)
model_VGG16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
epochs = 10
if os.path.exists('model_VGG16.h5'):
    model_VGG16 = tf.keras.models.load_model('model_VGG16.h5')
    print("Model loaded")

history = model_VGG16.fit(
    train_gen, 
    epochs=epochs,
    verbose = 1,
    validation_data=test_gen,
    callbacks = [es]
    )
model_VGG16.save('model_VGG16.h5') 

In [ ]:
model_VGG16 = tf.keras.models.load_model('model_VGG16.h5')
_, acc = model_VGG16.evaluate(valid_gen, verbose=0)
print('> %.3f' % (acc * 100.0))

## VGG19

In [ ]:
shape = (224, 224)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)

from keras.applications.vgg19 import VGG19

base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3)) 
 
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(1000, activation='relu')(x) # number of layers and units are hyperparameters, as usual
predictions = Dense(13, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy']) 

In [ ]:
epochs = 10
if os.path.exists('model_VGG19.h5'):
    model = tf.keras.models.load_model('model_VGG19.h5')
    print("Model loaded")
    
history = model.fit(
train_gen, 
epochs=epochs,
verbose = 1,
validation_data=test_gen,
callbacks = [es]
)
model.save('model_VGG19.h5') 

In [ ]:
model = tf.keras.models.load_model('model_VGG19.h5')
_, acc = model.evaluate(valid_gen, verbose=0)
print('> %.3f' % (acc * 100.0))

## InceptionV3

In [ ]:
shape = (150, 150)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)

from tensorflow.keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(input_shape = (150, 150, 3), include_top = False, weights = 'imagenet')

# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(1000, activation='relu')(x) # number of layers and units are hyperparameters, as usual
predictions = Dense(13, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model_InceptionV3 = Model(inputs=base_model.input, outputs=predictions)
model_InceptionV3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
epochs = 10
if os.path.exists('model_InceptionV3.h5'):
    model_InceptionV3 = tf.keras.models.load_model('model_InceptionV3.h5')
    print("Model loaded")

history = model_InceptionV3.fit(
    train_gen, 
    epochs=epochs,
    verbose = 1,
    validation_data=test_gen,
    callbacks = [es]
    )
model_InceptionV3.save('model_InceptionV3.h5') 

In [ ]:
model_InceptionV3 = tf.keras.models.load_model('model_InceptionV3.h5')
_, acc = model_InceptionV3.evaluate(valid_gen, verbose=0)
print('> %.3f' % (acc * 100.0))

## ResNet50

In [ ]:
shape = (224, 224)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)

from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(1000, activation='relu')(x) # number of layers and units are hyperparameters, as usual
predictions = Dense(13, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy']) 

In [ ]:
epochs = 10
if os.path.exists('model_ResNet50.h5'):
    model = tf.keras.models.load_model('model_ResNet50.h5')
    print("Model loaded")
    
history = model.fit(
    train_gen, 
    epochs=epochs,
    verbose = 1,
    validation_data=test_gen,
    callbacks = [es]
    )
model.save('model_ResNet50.h5') 

In [ ]:
model = tf.keras.models.load_model('model_ResNet50.h5')
_, acc = model.evaluate(valid_gen, verbose=0)
print('> %.3f' % (acc * 100.0))

## EfficientNet

In [ ]:
# !pip install -U efficientnet

# import efficientnet
# from efficientnet.keras import EfficientNetB0
from tensorflow.keras.applications.efficientnet import EfficientNetB0

shape = (224, 224)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)

base_model = EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(1000, activation='relu')(x) # number of layers and units are hyperparameters, as usual
predictions = Dense(13, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
epochs = 10
if os.path.exists('model_EfficientNet.h5'):
    model = tf.keras.models.load_model('model_EfficientNet.h5')
    print("Model loaded")

history = model.fit(
    train_gen, 
    epochs=epochs,
    verbose = 1,
    validation_data=test_gen,
    callbacks = [es]
    )
model.save('model_EfficientNet.h5') 

In [ ]:
model = tf.keras.models.load_model('model_EfficientNet.h5')
_, acc = model.evaluate(valid_gen, verbose=0)
print('> %.3f' % (acc * 100.0))

## EfficientNetV2L

In [ ]:
!pip install tensorflow==2.8

In [ ]:
# !pip install -U efficientnet

# import efficientnet
# from efficientnet.keras import EfficientNetB0
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2L

shape = (224, 224)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)

base_model = EfficientNetV2L(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(1000, activation='relu')(x) # number of layers and units are hyperparameters, as usual
predictions = Dense(13, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
epochs = 10
if os.path.exists('model_EfficientNetV2L.h5'):
    model = tf.keras.models.load_model('model_EfficientNetV2L.h5')
    print("Model loaded")
    
history = model.fit(
    train_gen, 
    epochs=epochs,
    verbose = 1,
    validation_data=test_gen,
    callbacks = [es]
    )
model.save('model_EfficientNetV2L.h5') 

In [ ]:
model = tf.keras.models.load_model('model_EfficientNetV2L.h5')
_, acc = model.evaluate(valid_gen, verbose=0)
print('> %.3f' % (acc * 100.0))

## Xception

In [ ]:
from tensorflow.keras.applications.xception import Xception

shape = (224, 224)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)

base_model = Xception(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(1000, activation='relu')(x) # number of layers and units are hyperparameters, as usual
predictions = Dense(13, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
epochs = 5
if os.path.exists('model_Xception.h5'):
    model = tf.keras.models.load_model('model_Xception.h5')
    print("Model loaded")

history = model.fit(
    train_gen, 
    epochs=epochs,
    verbose = 1,
    validation_data=test_gen,
    callbacks = [es]
    )
model.save('model_Xception.h5') 

In [ ]:
model = tf.keras.models.load_model('model_Xception.h5')
_, acc = model.evaluate(valid_gen, verbose=0)
print('> %.3f' % (acc * 100.0))

## DenseNet

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet201

shape = (224, 224)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)

base_model = DenseNet201(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(1000, activation='relu')(x) # number of layers and units are hyperparameters, as usual
predictions = Dense(13, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model_DenseNet201 = Model(inputs=base_model.input, outputs=predictions)
model_DenseNet201.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
epochs = 5
if os.path.exists('model_DenseNet201.h5'):
    model_DenseNet201 = tf.keras.models.load_model('model_DenseNet201.h5')
    print("Model loaded")

history = model_DenseNet201.fit(
    train_gen, 
    epochs=epochs,
    verbose = 1,
    validation_data=test_gen,
    callbacks = [es]
    )
model_DenseNet201.save('model_DenseNet201.h5')

In [ ]:
model_DenseNet201 = tf.keras.models.load_model('model_DenseNet201.h5')
_, acc = model_DenseNet201.evaluate(valid_gen, verbose=0)
print('> %.3f' % (acc * 100.0))

## Ensemble learning

In [ ]:
!pip install fbpca

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet201
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from tqdm import tqdm
import fbpca

shape = (224, 224)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)

base_model = DenseNet201(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
predictions = Flatten()(x)  # flatten from convolution tensor output  

model_DenseNet201 = Model(inputs=base_model.input, outputs=predictions)

######## Getting principal axes ########
train_gen.reset()
X_train, y_train = next(train_gen)
X_trainFeature = model_DenseNet201.predict(X_train)
for i in tqdm(range(7)): #1st batch is already fetched before the for loop.
    img, label = next(train_gen)
    X_trainFeature = np.append(X_trainFeature, model_DenseNet201.predict(np.array(img)), axis = 0)
    y_train = np.append(y_train, label, axis=0)
print(X_trainFeature.shape, y_train.shape)

mean = np.mean(X_trainFeature, axis = 0)
X_trainFeature_centered = X_trainFeature - mean
(U, s, Va) = fbpca.pca(X_trainFeature_centered, 500, True)
principalAxes = Va.T

# getting X_train, y_train, X_test, y_test from the generators
train_gen.reset()
X_train, y_train = next(train_gen)
X_trainFeature = (model_DenseNet201.predict(X_train)-mean).dot(principalAxes)
for i in tqdm(range(len(train_gen)-1)): #1st batch is already fetched before the for loop.
    img, label = next(train_gen)
    X_trainFeature = np.append(X_trainFeature, (model_DenseNet201.predict(np.array(img))-mean).dot(principalAxes), axis = 0)
    y_train = np.append(y_train, label, axis=0)
print(X_trainFeature.shape, y_train.shape)

test_gen.reset()
X_test, y_test = next(test_gen)
X_testFeature = (model_DenseNet201.predict(X_test)-mean).dot(principalAxes)
for i in tqdm(range(len(test_gen)-1)): #1st batch is already fetched before the for loop.
    img, label = next(test_gen)
    X_testFeature = np.append(X_testFeature, (model_DenseNet201.predict(np.array(img))-mean).dot(principalAxes), axis = 0)
    y_test = np.append(y_test, label, axis=0)
print(X_testFeature.shape, y_test.shape)

# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# le.fit(['B', 'K', 'N', 'P', 'Q', 'R', '_', 'b', 'k', 'n', 'p', 'q', 'r'])

In [ ]:
y_train = y_train.argmax(axis = 1)
y_test = y_test.argmax(axis = 1)

In [ ]:
def explainedVarianceSVD(thresh, arr):
    import numpy as np

    arr = np.square(arr)
    l = []
    i = 0
    csum = 0
    total = np.sum(arr)

    while csum/total < thresh and i < len(arr):
        csum += arr[i]
        l.append(csum/total)
        i += 1
    print(i)
    return l

In [ ]:
explainedVarianceSVD(.9, s)

In [ ]:
os.remove("X_train.csv")
os.remove("y_train.csv")
os.remove("X_test.csv")
os.remove("y_test.csv")

In [ ]:
pd.DataFrame(X_trainFeature).to_csv("X_train.csv")
pd.DataFrame(y_train).to_csv("y_train.csv")
pd.DataFrame(X_testFeature).to_csv("X_test.csv")
pd.DataFrame(y_test).to_csv("y_test.csv")

In [ ]:
X_train = np.array(pd.read_csv("X_train.csv"))
y_train = np.array(pd.read_csv("y_train.csv"))
X_test = np.array(pd.read_csv("X_test.csv"))
y_test = np.array(pd.read_csv("y_test.csv"))

from sklearn.ensemble import RandomForestClassifier
######## Random forest ########
randomForest = RandomForestClassifier()
randomForest.fit(X_train, y_train)
y_pred = randomForest.predict(X_test)

from sklearn import metrics
print("Train accuracy score =", metrics.accuracy_score(y_train, randomForest.predict(X_train)))
print("Accuracy score =", metrics.accuracy_score(y_test, y_pred))